<a href="https://colab.research.google.com/github/AstroNoodles/Mini-Projects/blob/master/RNNTweetGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Network With Trump Generator Tweets in Keras

Rnns to use:


*   GRU
*   LSTM
*   AlphaDropout / Dropout - To apply to the layer





In [0]:
# Import the libraries
import tensorflow.keras as keras
import pandas as pd


In [0]:
csv_url = "https://raw.githubusercontent.com/lucamusk/Atomhacks2019TrumpBot/master/TrumpTweets.csv?token=AXOm-jThREnKNDIWCp2--ddMRzO1C2VWks5cqMiXwA%3D%3D"
df = pd.read_csv(csv_url, header=0)

tweets = df['text']


In [0]:
train_set_tweets = tweets[:1000]
train_set_tweets = train_set_tweets.str.replace(" &amp;", "&")
tweets.head(5)

In [0]:
concat_thousand_tweets = ""
for i in range(1000):
  concat_thousand_tweets += train_set_tweets[i]
  
vocab = sorted(set(concat_thousand_tweets))

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
